# Test Anthropic Python SDK - Tool Use

This code example demonstrates tool use. It is inspired by a code sample from the Anthropic offical cookbook: https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/calculator_tool.ipynb

## Environment

```sh
pip install anthropic python-dotenv
```

In [19]:
from dotenv import load_dotenv
load_dotenv()

import anthropic

client = anthropic.Anthropic()

def get_weather(city: str) -> str:
    """Get the weather in a given city"""
    return f"The weather in {city} is sunny."

get_weather_metadata = {
    "name": "get_weather",
    "description": "Get the weather in a given city",
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the weather for"
            }
        },
        "required": ["city"]
    }
}

tools = [get_weather_metadata]

In [20]:
messages = [
  {
    "role": "user",
    "content": "What is the weather in Beijing?"
  }
]

message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024, # max_tokens is the maxium tokens to generate, it is required
    messages=messages,
    tools=tools
)

In [21]:
# Get the text output from the message
content = message.content
text = content[0].text
print(text)

I'll check the weather in Beijing for you.


In [22]:
message

Message(id='msg_01LXqSwzgoek7cxfMwkXdPSE', content=[TextBlock(citations=None, text="I'll check the weather in Beijing for you.", type='text'), ToolUseBlock(id='toolu_01Wm9dNNifPV2UmnE1X4K8wo', input={'city': 'Beijing'}, name='get_weather', type='tool_use')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=394, output_tokens=64, server_tool_use=None, service_tier='standard'))

In [23]:
# Get the tool use from the message
if message.stop_reason == "tool_use":
    for block in message.content:
        if block.type == "tool_use":
            print(block)
else:
    print("No tool use")

ToolUseBlock(id='toolu_01Wm9dNNifPV2UmnE1X4K8wo', input={'city': 'Beijing'}, name='get_weather', type='tool_use')


In [24]:
# Add the message content to the message history
messages.append({
  "role": "assistant",
  "content": content
})

In [25]:
# Call the tool
# How we handle tool use results in the message history is from the code sample:
# https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/calculator_tool.ipynb
if message.stop_reason == "tool_use":
    tool_use_content = []
    for block in message.content:
        if block.type == "tool_use":
            tool_use_content.append({
                "type": "tool_result",
                "tool_use_id": block.id,
                "content": get_weather(block.input["city"])
            })
    if len(tool_use_content) > 0:
        # We can treat multiple tool use results as a single user message
        messages.append({
            "role": "user",
            "content": tool_use_content
        })
else:
    print("No tool use")

In [26]:
message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024, # max_tokens is the maxium tokens to generate, it is required
    messages=messages,
    tools=tools
)

In [27]:
# Get the text output from the message
content = message.content
text = content[0].text
print(text)

The weather in Beijing is currently sunny.
